# Joining Data
- get data spread across multiple tables.

### SELECT t1.col_name, col2_ name t2.col_name, col2_name
### FROM table1 AS t1
### INNER JOIN table2 AS t2
###   ON (t1.col_name = t2.col_name)

- Join between (only) TWO tables: Purchasing.ProductVendor and Purchasing.Vendor.
- As always SQL Query vs Pandas.

## 1. Establish DB connection and...
- def funct. 'df_from_SQL(qry)' to convert SQL Query result to DF (I define df_from_query() function to avoid using pd.read_sql() cause the warning message when using pyodbc).
- def funct. 'jm_sample()' to show a fixed sample of the df
- Vamos a tener que pensar una función que compare df o directamente que me imprima las diferencias entre dos o más df: df.compare()

In [28]:
### Connect to the DB - Establish the connection
import pyodbc

# Valid values for the connection string
driver = '{ODBC Driver 17 for SQL Server}'
server = '(local)'
dbname = 'AdventureWorks2019'
user = 'user1'
passwd = 'pass1'

# Construct the Connection String
connection_string = f'DRIVER={driver};SERVER={server};\
    DATABASE={dbname};UID={user};PWD={passwd}'
print('Connection String:\n', connection_string)

# Establish the connection
try:
    cnx = pyodbc.connect(connection_string)
    cur = cnx.cursor()
except pyodbc.Error as e:
    print('ERROR:', e)
else:
    print('SUCCESS: Connection Established')

# mk function to convert SQL queries to DF
import pandas as pd

def df_from_query(qry):     # convert cursor.execute(query) to DF
    cur.execute(qry)
    field_names = [i[0] for i in cur.description]
    get_data = [list(x) for x in cur]
    df = pd.DataFrame(data=get_data, columns=field_names)
    return df

def df_view(df, n=6):
    l6 = [0, 4, 9, -9, -5, -1]
    l4 = [0, 2, -3, -1]
    if n == 6:
        lst = l6
    else:
        lst = l4

    display(df.info())
    display(df.iloc[[0, 4, 9, -9, -5, -1]])


Connection String:
 DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);    DATABASE=AdventureWorks2019;UID=user1;PWD=pass1
SUCCESS: Connection Established


## 2. Get both tables as a DF: t1_df, t2_df
- To apply native pandas code i need tables as dfs.

In [29]:
query = ''' SELECT * FROM Purchasing.ProductVendor'''
t1_df = df_from_query(query)

df_view(t1_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ProductID         460 non-null    int64         
 1   BusinessEntityID  460 non-null    int64         
 2   AverageLeadTime   460 non-null    int64         
 3   StandardPrice     460 non-null    object        
 4   LastReceiptCost   460 non-null    object        
 5   LastReceiptDate   460 non-null    datetime64[ns]
 6   MinOrderQty       460 non-null    int64         
 7   MaxOrderQty       460 non-null    int64         
 8   OnOrderQty        155 non-null    float64       
 9   UnitMeasureCode   460 non-null    object        
 10  ModifiedDate      460 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 39.7+ KB


None

,ProductID,BusinessEntityID,AverageLeadTime,StandardPrice,LastReceiptCost,LastReceiptDate,MinOrderQty,MaxOrderQty,OnOrderQty,UnitMeasureCode,ModifiedDate
0,1,1580,17,47.8700,50.2635,2011-08-29,1,5,3.0,CS,2011-08-29
4,317,1678,17,25.7700,27.0585,2011-08-25,100,1000,NaN,EA,2011-08-25
9,319,1678,17,43.8700,46.0635,2011-08-25,100,1000,NaN,EA,2011-08-25
451,937,1638,18,59.9900,62.9895,2011-08-27,100,1000,300.0,EA,2011-08-27
455,939,1680,16,45.9900,48.2895,2011-08-27,100,1000,NaN,EA,2011-08-27
459,952,1674,19,14.9900,15.7395,2011-08-29,20,100,60.0,DZ,2011-08-29


In [30]:
query = ''' SELECT * FROM Purchasing.Vendor'''
t2_df = df_from_query(query)

df_view(t2_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   BusinessEntityID         104 non-null    int64         
 1   AccountNumber            104 non-null    object        
 2   Name                     104 non-null    object        
 3   CreditRating             104 non-null    int64         
 4   PreferredVendorStatus    104 non-null    bool          
 5   ActiveFlag               104 non-null    bool          
 6   PurchasingWebServiceURL  6 non-null      object        
 7   ModifiedDate             104 non-null    datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(2), object(3)
memory usage: 5.2+ KB


None

,BusinessEntityID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,ModifiedDate
0,1492,AUSTRALI0001,Australia Bike Retailer,1,True,True,None,2011-12-23
4,1500,MORGANB0001,Morgan Bike Accessories,1,True,True,None,2012-02-02
9,1510,INTERNAT0001,International,1,True,True,None,2012-01-25
95,1682,PREMIER0001,"Premier Sport, Inc.",1,True,True,None,2012-02-02
99,1690,BLOOMING0001,Bloomington Multisport,1,True,True,None,2011-12-23
103,1698,BUSINESS0001,Business Equipment Center,2,True,True,None,2011-12-23


## 3. First Question - First JOIN
- What are the supplier name of the products?

### 3.1. SQL direct to the DB

In [31]:
q31 = ''' SELECT t1.ProductID, StandardPrice, t2.BusinessEntityID, Name
        FROM Purchasing.ProductVendor AS t1
        INNER JOIN Purchasing.Vendor AS t2
            ON (t1.BusinessEntityID = t2.BusinessEntityID)'''
q31_df = df_from_query(q31)

df_view(q31_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ProductID         460 non-null    int64 
 1   StandardPrice     460 non-null    object
 2   BusinessEntityID  460 non-null    int64 
 3   Name              460 non-null    object
dtypes: int64(2), object(2)
memory usage: 14.5+ KB


None

,ProductID,StandardPrice,BusinessEntityID,Name
0,1,47.8700,1580,"Litware, Inc."
4,317,25.7700,1678,"Proseware, Inc."
9,319,43.8700,1678,"Proseware, Inc."
451,937,59.9900,1638,Inline Accessories
455,939,45.9900,1680,Jackson Authority
459,952,14.9900,1674,Varsity Sport Co.


## 3.2. Native Pandas, (I saw) three tools: 
1. join: combining data on a column or index. - Is an 'instance method': df.join()
2. merge: combining data on common columns or indices. - Is a 'module function': pd.merge()
3. concat: combining data across rows or columns. - Module Function. Simpler way to combine (add) datasets by rows or by columns (axis='columnns)
- https://realpython.com/pandas-merge-join-and-concat/

In [32]:
## df.join()
dfi = t1_df.join(
    t2_df.set_index(['BusinessEntityID']),
    on=['BusinessEntityID'],
    how='inner',
    lsuffix='_t1',
    rsuffix='_t2')[['ProductID', 'StandardPrice', 'BusinessEntityID',
                    'Name']].sort_values(by=['ProductID', 'BusinessEntityID'])

q32_1_df = dfi.reset_index(drop=True)

print(f'''SQL result equal df.join() result: {q31_df.equals(q32_1_df)}\n''')
# wo/.sort_values() the result data IS THE SAME but diff. row order
#print(q31_df.compare(q32_1_df))

df_view(q32_1_df)

SQL result equal df.join() result: True

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ProductID         460 non-null    int64 
 1   StandardPrice     460 non-null    object
 2   BusinessEntityID  460 non-null    int64 
 3   Name              460 non-null    object
dtypes: int64(2), object(2)
memory usage: 14.5+ KB


None

,ProductID,StandardPrice,BusinessEntityID,Name
0,1,47.8700,1580,"Litware, Inc."
4,317,25.7700,1678,"Proseware, Inc."
9,319,43.8700,1678,"Proseware, Inc."
451,937,59.9900,1638,Inline Accessories
455,939,45.9900,1680,Jackson Authority
459,952,14.9900,1674,Varsity Sport Co.


In [33]:
## pd.merge()
q32_2_df = pd.merge(t1_df, t2_df,
         on=['BusinessEntityID'])[['ProductID', 'StandardPrice',
                                   'BusinessEntityID', 'Name']]

q32_2_df.sort_values(by=['ProductID', 'BusinessEntityID'], inplace=True)
q32_2_df.reset_index(drop=True, inplace=True)
print(q31_df.equals(q32_2_df))
print(q31_df.compare(q32_2_df))


True
Empty DataFrame
Columns: []
Index: []


## 4. Second Question
- return the product and supplier information for any combination of parts supplied by a company for which the company name starts with the letter F and the price of the product is more than $10.

### 4.1. Direct SQL to DB

In [34]:
q41 = ''' SELECT t1.ProductID, t1.StandardPrice, t2.BusinessEntityID, t2.Name AS SupplierName
          FROM Purchasing.ProductVendor AS t1
          INNER JOIN Purchasing.Vendor AS t2
            ON (t1.BusinessEntityID = t2.BusinessEntityID)
          WHERE StandardPrice > $10 AND Name LIKE N'F%'; '''
q41_df = df_from_query(q41)
df_view(q41_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ProductID         12 non-null     int64 
 1   StandardPrice     12 non-null     object
 2   BusinessEntityID  12 non-null     int64 
 3   SupplierName      12 non-null     object
dtypes: int64(2), object(2)
memory usage: 512.0+ bytes


None

,ProductID,StandardPrice,BusinessEntityID,SupplierName
0,908,20.0900,1570,First Rate Bicycles
4,850,24.7500,1594,Fitness Association
9,855,38.5000,1594,Fitness Association
3,849,24.7500,1594,Fitness Association
7,853,32.3500,1594,Fitness Association
11,857,37.5000,1594,Fitness Association


## 4.2 Pandas df.join() and pd.merge()

In [35]:
## df.join()
dfi = t1_df.join(
    t2_df.set_index(['BusinessEntityID']),
    on=['BusinessEntityID'],
    how='inner',
    lsuffix='_t1',
    rsuffix='_t2')[['ProductID', 'StandardPrice', 'BusinessEntityID',
                    'Name']].rename(columns={'Name': 'SupplierName'})

q42_1_df = dfi[(dfi.StandardPrice > 10) &
              (dfi.SupplierName.str.startswith('F'))].sort_values(by='SupplierName')
q42_1_df.reset_index(drop=True, inplace=True)

#df_view(q42_1_df)
q41_df.equals(q42_1_df)


True

In [36]:
## pd.merge()
q42_2_df = pd.merge(t1_df, t2_df,
         on=['BusinessEntityID'])[['ProductID', 'StandardPrice',
                                   'BusinessEntityID', 'Name']]

q32_2_df.sort_values(by=['ProductID', 'BusinessEntityID'], inplace=True)
q32_2_df.reset_index(drop=True, inplace=True)
print(q31_df.equals(q32_2_df))
print(q31_df.compare(q32_2_df))

True
Empty DataFrame
Columns: []
Index: []


In [37]:
dfi2 = t1_df.merge(t2_df, on=['BusinessEntityID'],
                   how='inner')[['ProductID', 'StandardPrice','BusinessEntityID',
                                 'Name']].rename(columns={'Name': 'SupplierName'})

q42_2_df = dfi2[(dfi2.StandardPrice > 10) &
                (dfi2.SupplierName.str.startswith('F'))].sort_values(by='SupplierName')
q42_2_df.reset_index(drop=True, inplace=True)

#df_view(q42_1_df)
q41_df.equals(q42_2_df)

True

## 5. Third Question
- ¿How many products per Supplier?
- write a query that uses information in both tables to determine how many products were supplied by each supplier.
- Es simple cuento cuantos SupplierName tengo en el join agrupados por SupplierName porque c/linea de ese resultado es un producto

## 5.1. Direct SQL to the DB

In [42]:
q51 = ''' SELECT t2.Name AS SupplierName, COUNT(1) AS ProductsNumber
        FROM Purchasing.ProductVendor AS t1
        INNER JOIN Purchasing.Vendor AS t2
            ON (t1.BusinessEntityID = t2.BusinessEntityID)
        GROUP BY t2.Name
        ORDER BY ProductsNumber DESC, SupplierName'''

q51_df = df_from_query(q51)

df_view(q51_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SupplierName    86 non-null     object
 1   ProductsNumber  86 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


None

,SupplierName,ProductsNumber
0,Mountain Works,30
4,Aurora Bike Center,19
9,Ready Rentals,16
77,"Midwest Sport, Inc.",1
81,Sport Playground,1
85,Wood Fitness,1


## 5.2. Native Pandas w/ join and merge
- at this points the steps are simple, get the DF from a join/merge and then mk the count as in 3_...ipynb using value_counts or groupby + count.

In [76]:
## df.join()
dfi = t1_df.join(
    t2_df.set_index(['BusinessEntityID']),
    on=['BusinessEntityID'],
    how='inner',
    lsuffix='_t1',
    rsuffix='_t2')[['ProductID', 'StandardPrice', 'BusinessEntityID',
                    'Name']].rename(columns={'Name': 'SupplierName'})

q52_df = dfi.value_counts('SupplierName').reset_index()
# SN = dfi.value_counts('SupplierName')
# q52_df = pd.DataFrame({'SupplierName': SN.index, 'ProductsNumber': SN.values})
q52_df.rename(columns={'count': 'ProductsNumber'}, inplace=True)
q52_df.sort_values(['ProductsNumber', 'SupplierName'], ascending=[False, True],
                   inplace=True)
#q52_df.index = pd.RangeIndex(start=0, stop=86, step=1)

df_view(q52_df)

q51_df.equals(q52_df)

## OJO acá varias cosas para la comparación 
# 1. != types of indexes (RangeIndex, int Index, int64.. etc)
# 2. Revisar el 'creterio' de orden algabético:
# 52	Signature Cycles	SUPERSALES INC.
# 53	Superior Bicycles	Signature Cycles
# 54	SUPERSALES INC.	Superior Bicycles

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SupplierName    86 non-null     object
 1   ProductsNumber  86 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


None

,SupplierName,ProductsNumber
0,Mountain Works,30
4,Aurora Bike Center,19
9,Ready Rentals,16
77,"Midwest Sport, Inc.",1
81,Sport Playground,1
85,Wood Fitness,1


False

In [79]:
q51_df.compare(q52_df)
# display(q51_df.head(5))
# display(q52_df.head(5))
# print(q51_df.columns)
# print(q52_df.columns)
# print(q51_df.columns == q52_df.columns)
# print(q51_df.index, type(q51_df.index))
# print(q52_df.index, type(q52_df.index))


SupplierName                   
                 self              other
52   Signature Cycles    SUPERSALES INC.
53  Superior Bicycles   Signature Cycles
54    SUPERSALES INC.  Superior Bicycles

In [80]:
for df in (q51_df, q52_df):
    display(df.iloc[52:55])


,SupplierName,ProductsNumber
52,Signature Cycles,2
53,Superior Bicycles,2
54,SUPERSALES INC.,2


,SupplierName,ProductsNumber
52,SUPERSALES INC.,2
53,Signature Cycles,2
54,Superior Bicycles,2


### groupby + count and merge is similar...
- No las voy a desarrollar ahora, paso a 6

## 6. An 'EXTERNAL' exercises
1. Make an SQL inner join to get ProductID - ProductName + BusinessEntityID table, in order to have in the above queries not only the productID but also the productName. Put in t3_df
2. The first part with SQL (as i said in 1.), and the second part with Pandas using the early t2_df (join and merge betwwen the new t3_df and t2_df )
3. To che correct results, sort by the correct values and compare several column of q31 and this new q6_df
4. SQL CTE: you can do all this in native SQL using CTE, something i have to analyze specially in MSSQL Srv and exercise in early ntbk (5_AS_&_WITH)

In [87]:
## t3_df: new df that is de necessary join to get ProductID and ProductName
q6 = ''' select t1.ProductID, t2.Name AS ProdName, t1. StandardPrice, BusinessEntityID
            from Purchasing.ProductVendor as t1
            inner join Production.Product as t2
	            on (t1.ProductID = t2.ProductID);'''

t3_df = df_from_query(q6)

t3_df

,ProductID,ProdName,StandardPrice,BusinessEntityID
0,1,Adjustable Race,47.8700,1580
1,2,Bearing Ball,39.9200,1688
2,4,Headset Ball Bearings,54.3100,1650
3,317,LL Crankarm,28.1700,1578
4,317,LL Crankarm,25.7700,1678
...,...,...,...,...
455,939,ML Road Pedal,45.9900,1680
456,940,HL Road Pedal,59.9900,1508
457,941,Touring Pedal,59.9900,1628
458,948,Front Brakes,78.8900,1576


In [89]:
dfi = t3_df.merge(t2_df, on=['BusinessEntityID'],
                   how='inner')[['ProductID', 'ProdName', 'StandardPrice',
                                 'BusinessEntityID', 'Name']].rename(
                                     columns={'Name': 'SupplierName'})

q6_df = dfi[(dfi.StandardPrice > 10) &
            (dfi.SupplierName.str.startswith('F'))].sort_values(by='SupplierName')
q6_df.reset_index(drop=True, inplace=True)

df_view(q6_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ProductID         12 non-null     int64 
 1   ProdName          12 non-null     object
 2   StandardPrice     12 non-null     object
 3   BusinessEntityID  12 non-null     int64 
 4   SupplierName      12 non-null     object
dtypes: int64(2), object(3)
memory usage: 608.0+ bytes


None

,ProductID,ProdName,StandardPrice,BusinessEntityID,SupplierName
0,908,LL Mountain Seat/Saddle,20.0900,1570,First Rate Bicycles
4,850,"Men's Sports Shorts, L",24.7500,1594,Fitness Association
9,855,"Men's Bib-Shorts, S",38.5000,1594,Fitness Association
3,849,"Men's Sports Shorts, M",24.7500,1594,Fitness Association
7,853,"Women's Tights, M",32.3500,1594,Fitness Association
11,857,"Men's Bib-Shorts, L",37.5000,1594,Fitness Association
